In [ ]:
import numpy as np

from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Input, LSTM, Dense
from keras.utils import to_categorical

# Подготовка обучающих данных

## Текст на русском

In [ ]:
path_to_text_rus = '/content/drive/MyDrive/polytech/ГЭ/TheHungerGames_rus.txt'
with open(path_to_text_rus, 'r', encoding='utf-8') as file:
    text_rus = file.read() \
                   .lower() \
                   .replace('\n', ' ')

In [ ]:
chars_rus = sorted(list(set(text_rus)))
dict_chars = dict((chars_rus[i], i) for i in range(len(chars_rus)))

print(f'Количество уникальных символов: {len(chars_rus)}\n')
print(f'Символы:')
columns = 5
rows = (len(dict_chars) + columns - 1) // columns
for i in range(rows):
    row = []
    for j in range(columns):
        index = i + j * rows
        if index < len(dict_chars):
            key = list(dict_chars.keys())[index]
            value = dict_chars[key]
            row.append(f'{value + 1}) \'{key}\'')
    print('\t\t'.join(row))

Количество уникальных символов: 48

Символы:
1) ' '		11) '?'		21) 'з'		31) 'с'		41) 'ы'
2) '!'		12) '«'		22) 'и'		32) 'т'		42) 'ь'
3) ','		13) '»'		23) 'й'		33) 'у'		43) 'э'
4) '-'		14) 'а'		24) 'к'		34) 'ф'		44) 'ю'
5) '.'		15) 'б'		25) 'л'		35) 'х'		45) 'я'
6) '1'		16) 'в'		26) 'м'		36) 'ц'		46) 'ё'
7) '2'		17) 'г'		27) 'н'		37) 'ч'		47) '–'
8) '3'		18) 'д'		28) 'о'		38) 'ш'		48) '…'
9) ':'		19) 'е'		29) 'п'		39) 'щ'
10) ';'		20) 'ж'		30) 'р'		40) 'ъ'


In [ ]:
# Параметры формирования выборок
max_len = 60
step = 3
# Текстовое представление выборок
sentences = []
next_chars = []

print('Выборки:')
cnt = 0
for i in range(0, len(text_rus) - max_len, step):
    sentences.append(text_rus[i: i + max_len])
    next_chars.append(text_rus[i + max_len])
    if cnt < 20:
        print(f'{cnt + 1}) {sentences[cnt]} -> \'{next_chars[cnt]}\'')
        cnt += 1

Выборки:
1) я просыпаюсь и чувствую, что рядом на кровати пусто. пытаюсь -> ' '
2) росыпаюсь и чувствую, что рядом на кровати пусто. пытаюсь на -> 'щ'
3) ыпаюсь и чувствую, что рядом на кровати пусто. пытаюсь нащуп -> 'а'
4) юсь и чувствую, что рядом на кровати пусто. пытаюсь нащупать -> ' '
5)  и чувствую, что рядом на кровати пусто. пытаюсь нащупать те -> 'п'
6) чувствую, что рядом на кровати пусто. пытаюсь нащупать тепло -> ' '
7) ствую, что рядом на кровати пусто. пытаюсь нащупать тепло пр -> 'и'
8) ую, что рядом на кровати пусто. пытаюсь нащупать тепло прим, -> ' '
9)  что рядом на кровати пусто. пытаюсь нащупать тепло прим, но -> ' '
10) о рядом на кровати пусто. пытаюсь нащупать тепло прим, но по -> 'д'
11) ядом на кровати пусто. пытаюсь нащупать тепло прим, но под п -> 'а'
12) м на кровати пусто. пытаюсь нащупать тепло прим, но под паль -> 'ц'
13) а кровати пусто. пытаюсь нащупать тепло прим, но под пальцам -> 'и'
14) ровати пусто. пытаюсь нащупать тепло прим, но под пальцами л

In [ ]:
X = np.zeros((len(sentences), max_len, len(chars_rus)), dtype=bool)
y = np.zeros((len(sentences), len(chars_rus)), dtype=bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, dict_chars[char]] = 1
    y[i, dict_chars[next_chars[i]]] = 1

## Текст на английском

In [ ]:
path_to_text_eng = '/content/drive/MyDrive/polytech/ГЭ/TheHungerGames_eng.txt'
with open(path_to_text_eng, 'r', encoding='utf-8') as file:
    text_eng = file.read() \
                   .lower() \
                   .replace('\n', ' ')

# Определение модели

In [ ]:
model = Sequential()
model.add(Input(shape=(max_len, len(chars_rus))))
model.add(LSTM(128))
model.add(Dense(len(chars_rus), activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_1 (LSTM)                        │ (None, 128)                 │          90,624 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 48)                  │           6,192 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 96,816 (378.19 KB)

 Trainable params: 96,816 (378.19 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.fit(X, y, batch_size=128, epochs=20)

Epoch 1/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 12s 154ms/step - accuracy: 0.1401 - loss: 3.4954
Epoch 2/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 19s 134ms/step - accuracy: 0.1617 - loss: 3.1505
Epoch 3/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 10s 134ms/step - accuracy: 0.1684 - loss: 3.0936
Epoch 4/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 11s 152ms/step - accuracy: 0.1854 - loss: 2.9905
Epoch 5/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 20s 151ms/step - accuracy: 0.2086 - loss: 2.8782
Epoch 6/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 22s 180ms/step - accuracy: 0.2306 - loss: 2.7757
Epoch 7/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 18s 137ms/step - accuracy: 0.2478 - loss: 2.6842
Epoch 8/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 10s 133ms/step - accuracy: 0.2645 - loss: 2.6291
Epoch 9/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 11s 153ms/step - accuracy: 0.2687 - loss: 2.5634
Epoch 10/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 20s 152ms/step - accuracy: 0.2729 - loss: 2.5311
Epoch 11/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 20s 150ms/step - accuracy: 0.2683 - loss: 2.5217
Epoch 12/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 21

In [ ]:
class Letter2Vec:

    def __init__(self, text_path, max_len, step):
        self.max_len = max_len
        self.step = step

        self.__load_text(text_path)
        self.get_tokens()
        self.get_text_samples()
        self.get_vector_samples()
        self.define_model()

    def __load_text(self, path):
        with open(path, 'r', encoding='utf-8') as file:
            self.text = file.read() \
                            .lower() \
                            .replace('\n', ' ')

    def get_tokens(self, show_info=False):
        chars = sorted(list(set(self.text)))
        self.tokens = dict((chars[i], i) for i in range(len(chars)))
        if show_info:
            print(f'Токены:')
            olumns = 5
            rows = (len(self.tokens) + columns - 1) // columns
            for i in range(rows):
                row = []
                for j in range(columns):
                    index = i + j * rows
                    if index < len(self.tokens):
                        key = list(self.tokens.keys())[index]
                        value = self.tokens[key]
                        row.append(f'{value + 1}) \'{key}\'')
                print('\t\t'.join(row))

    def get_text_samples(self, show_info=False):
        self.sentences = []
        self.next_chars = []
        if show_info:
            print('Примеры выборок:')
            cnt = 0
        for i in range(0, len(self.text) - self.max_len, self.step):
            self.sentences.append(self.text[i: i + self.max_len])
            self.next_chars.append(self.text[i + self.max_len])
            if show_info:
                if cnt < 20:
                    print(f'{cnt + 1}) {self.sentences[cnt]} -> \'{self.next_chars[cnt]}\'')
                    cnt += 1

    def get_vector_samples(self, show_info=False):
        self.X = np.zeros((len(self.sentences), self.max_len, len(self.tokens)), dtype=bool)
        self.y = np.zeros((len(self.sentences), len(self.tokens)), dtype=bool)
        for i, sentence in enumerate(self.sentences):
            for t, char in enumerate(sentence):
                self.X[i, t, self.tokens[char]] = 1
            self.y[i, self.tokens[self.next_chars[i]]] = 1
        if show_info:
            print(f'X.shape: {self.X.shape}')
            print(f'y.shape: {self.y.shape}')

    def define_model(self, show_info=False):
        self.model = Sequential()
        self.model.add(Input(shape=(max_len, len(chars_rus))))
        self.model.add(LSTM(128))
        self.model.add(Dense(len(chars_rus), activation='softmax'))
        self.model.compile(loss='categorical_crossentropy',
                      optimizer='adam',
                      metrics=['accuracy'])
        if show_info:
            self.model.summary()

In [ ]:
letter2vec = Letter2Vec(path_to_text_rus, max_len=60, step=3)

In [ ]:
letter2vec.get_tokens(show_info=True)

Токены:
1) ' '		11) '?'		21) 'з'		31) 'с'		41) 'ы'
2) '!'		12) '«'		22) 'и'		32) 'т'		42) 'ь'
3) ','		13) '»'		23) 'й'		33) 'у'		43) 'э'
4) '-'		14) 'а'		24) 'к'		34) 'ф'		44) 'ю'
5) '.'		15) 'б'		25) 'л'		35) 'х'		45) 'я'
6) '1'		16) 'в'		26) 'м'		36) 'ц'		46) 'ё'
7) '2'		17) 'г'		27) 'н'		37) 'ч'		47) '–'
8) '3'		18) 'д'		28) 'о'		38) 'ш'		48) '…'
9) ':'		19) 'е'		29) 'п'		39) 'щ'
10) ';'		20) 'ж'		30) 'р'		40) 'ъ'


In [ ]:
letter2vec.get_text_samples(show_info=True)

Примеры выборок:
1) я просыпаюсь и чувствую, что рядом на кровати пусто. пытаюсь -> ' '
2) росыпаюсь и чувствую, что рядом на кровати пусто. пытаюсь на -> 'щ'
3) ыпаюсь и чувствую, что рядом на кровати пусто. пытаюсь нащуп -> 'а'
4) юсь и чувствую, что рядом на кровати пусто. пытаюсь нащупать -> ' '
5)  и чувствую, что рядом на кровати пусто. пытаюсь нащупать те -> 'п'
6) чувствую, что рядом на кровати пусто. пытаюсь нащупать тепло -> ' '
7) ствую, что рядом на кровати пусто. пытаюсь нащупать тепло пр -> 'и'
8) ую, что рядом на кровати пусто. пытаюсь нащупать тепло прим, -> ' '
9)  что рядом на кровати пусто. пытаюсь нащупать тепло прим, но -> ' '
10) о рядом на кровати пусто. пытаюсь нащупать тепло прим, но по -> 'д'
11) ядом на кровати пусто. пытаюсь нащупать тепло прим, но под п -> 'а'
12) м на кровати пусто. пытаюсь нащупать тепло прим, но под паль -> 'ц'
13) а кровати пусто. пытаюсь нащупать тепло прим, но под пальцам -> 'и'
14) ровати пусто. пытаюсь нащупать тепло прим, но под па

In [ ]:
letter2vec.get_vector_samples(show_info=True)

X.shape: (8608, 60, 48)
y.shape: (8608, 48)


In [ ]:
letter2vec.define_model(show_info=True)

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_3 (LSTM)                        │ (None, 128)                 │          90,624 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 48)                  │           6,192 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 96,816 (378.19 KB)

 Trainable params: 96,816 (378.19 KB)

 Non-trainable params: 0 (0.00 B)

In [102]:
!git config --global user.name "serjshul"
!git config --global user.email "shulginsergey0@gmail.com"

In [104]:
!cp /content/drive/MyDrive/polytech/ГЭ/ПЗвИС_ЛР3_Шульгин.ipynb /content/Text-Generation_LSTM

In [106]:
!git reset --hard HEAD~1

HEAD is now at eabc05c Initial commit


In [105]:
%cd /content/Text-Generation_LSTM

!git add .
!git commit -m "ПЗвИС_ЛР3_Шульгин.ipynb init"
!git push origin main

/content/Text-Generation_LSTM
[main 10feb5a] ПЗвИС_ЛР3_Шульгин.ipynb init
 1 file changed, 1 insertion(+)
 create mode 100644 "\320\237\320\227\320\262\320\230\320\241_\320\233\320\2403_\320\250\321\203\320\273\321\214\320\263\320\270\320\275.ipynb"
Enumerating objects: 4, done.
Counting objects: 100% (4/4), done.
Delta compression using up to 2 threads
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 5.44 KiB | 5.44 MiB/s, done.
Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
remote: error: GH013: Repository rule violations found for refs/heads/main.
remote: 
remote: - GITHUB PUSH PROTECTION
remote:   —————————————————————————————————————————
remote:     Resolve the following violations before pushing again
remote: 
remote:     - Push cannot contain secrets
remote: 
remote:     
remote:      (?) Learn how to resolve a blocked push
remote:      https://docs.github.com/code-security/secret-scanning/working-with-secret-scanning-and-push-protection/working-with-pus